In [13]:
import pandas as pd, numpy as np,re,os,glob
from datetime import datetime
pd.set_option('display.max_colwidth', 155)

In [15]:
tunTri=pd.read_csv('https://raw.githubusercontent.com/ExploitIdF/Referentiel_Tunnels/refs/heads/main/import/tunnelTricode.csv')
CorTbFerm=pd.read_csv('https://raw.githubusercontent.com/ExploitIdF/Referentiel_Tunnels/refs/heads/main/import/CorTunnelSensFerm.csv')
local=pd.read_csv('https://raw.githubusercontent.com/ExploitIdF/Referentiel_Tunnels/refs/heads/main/import/Local.csv')

In [26]:
#@title Création de la table des niches de sécurité et incendie
local=pd.read_csv('https://raw.githubusercontent.com/ExploitIdF/Referentiel_Tunnels/refs/heads/main/import/Local.csv')
niches=local[local['Type Famille']=='LOCAL-NICHE'][['Tatouage', 'Description', 'Code Exploitant',   'Axe', 'Sens',
       'Pr', 'État',  'Dans tunnel',    'Tunnel',  'Segment',  'Parent',  'GPS']].rename({'Code Exploitant':'CodeEx'},axis=1)
niches=niches[niches['Tunnel']!='LA_JONCHERE'].join(tunTri.set_index('Tunnel')['triCode'],on='Tunnel')
niches=niches.join(CorTbFerm.set_index(['triCode','Sens'])['Fermeture'],on=['triCode','Sens'])
niches.to_csv('niches.csv',index=False)
niches[niches['Fermeture'].isna()]
niches[niches['État'].str[0]!='E']

,Tatouage,Description,CodeEx,Axe,Sens,Pr,État,Dans tunnel,Tunnel,Segment,Parent,GPS,triCode,Fermeture
1661,L11.944T,Niche incendie de l'issue de secours n°423a,NS-423a/FON,A12,W,"5,0486",PROJET,Oui,FONTENAY,12016622,NaN,"48.805294,2.050994",FLF,FLF-W


In [17]:
issues=pd.read_csv('https://raw.githubusercontent.com/ExploitIdF/IssuesTunnels/main/_static/issuesTunnelsDIRIF.csv')
issuesFermetures=issues[['CodeEx', 'PC', 'Tatouage', 'triCode' ,'Sens']] .join(CorTbFerm.set_index(['triCode','Sens'])['Fermeture'],on=['triCode','Sens'])
issuesFermetures

,CodeEx,PC,Tatouage,triCode,Sens,Fermeture
0,IS101,PCO,L10.366R,A14,Y,A14&NEU-Y
1,IS102,PCO,L10.367S,A14,Y,A14&NEU-Y
2,IS103,PCO,L10.368T,A14,Y,A14&NEU-Y
3,IS104,PCO,L10.369U,A14,Y,A14&NEU-Y
4,IS105,PCO,L10.371W,A14,Y,A14&NEU-Y
...,...,...,...,...,...,...
285,IS463,PCO,L11.515H,SCL,W,BAP&SCL-W
286,IS421A,PCO,L11.916K,FLF,Y,FLF-Y
287,IS421B,PCO,L11.917L,FLF,Y,FLF-Y
288,IS423A,PCO,L11.918M,FLF,W,FLF-W


,Tatouage,Description,Code Exploitant,Axe,Sens,Pr,État,Dans tunnel,Tunnel,Segment,Parent,GPS,triCode
1661,L11.944T,Niche incendie de l'issue de secours n°423a,NS-423a/FON,A12,W,"5,0486",PROJET,Oui,FONTENAY,12016622,NaN,"48.805294,2.050994",FLF


,Tatouage,Description,Code Exploitant,Axe,Sens,Pr,État,Dans tunnel,Tunnel,Segment,Parent,GPS,triCode,Fermeture


In [23]:
for cl in niches.columns:
  print('null:',niches[cl].isna().sum())
  print(niches[cl].value_counts()[:6],'\n')

null: 0
Tatouage
L10.122A    1
L11.295W    1
L11.346H    1
L11.345G    1
L11.344F    1
L11.343E    1
Name: count, dtype: int64 

null: 0
Description
Niche sécurité NSW-701 - PAU NM01R - Tunnel de Nanterre-La Défense    1
Niche incendie NSW-004 Sortie Sud - Tunnel de Taverny                 1
Niche sécurité NSY-009 - PAU AL01F - Tunnel du Landy                  1
Niche sécurité NSY-101 - PAU AL01H - Tunnel du Landy                  1
Niche sécurité NSW-008 - PAU AL00R - Tunnel du Landy                  1
Niche sécurité NSW-006 - PAU AL01X - Tunnel du Landy                  1
Name: count, dtype: int64 

null: 0
Code Exploitant
DEF-NSW-701/DEF    1
NSW-004/TAV        1
NSY-009/LAN        1
NSY-101/LAN        1
NSW-008/LAN        1
NSW-006/LAN        1
Name: count, dtype: int64 

null: 0
Axe
A13        55
A14        50
A86NORD    49
A86EST     38
N19        32
A6B        28
Name: count, dtype: int64 

null: 0
Sens
Y      147
W      137
E       69
I       64
W>I      7
E>W      4
Name: coun